In [1]:
from datasets import load_dataset, DatasetDict, Dataset
squad: DatasetDict = load_dataset("squad")
squad_train: Dataset = squad['train']
squad_validation: Dataset = squad['validation']

In [17]:
import spacy

# Load the English NLP model
nlp = spacy.load("en_core_web_sm", disable=["parser", "tagger", "ner", "lemmatizer", "attribute_ruler"])
# nlp = spacy.load("en_core_web_sm")

In [29]:
len(squad_train['context'][0])

695

In [31]:
context_lengths = [len(context) for context in squad_train['context']]

import numpy as np

np.quantile(context_lengths, [0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

array([ 559. ,  693. ,  895. , 1147. , 1328.1, 1749. ])

In [18]:
def tokenize_example_spacy(example):
    # Tokenize context
    context_doc = nlp(example['context'])
    tokenized_context = [token.text for token in context_doc]
    context_token_char_offsets = [token.idx for token in context_doc]

    # Tokenize question
    question_doc = nlp(example['question'])
    tokenized_question = [token.text for token in question_doc]

    # --- Crucial Step: Map Answer Spans (Simplified) ---
    # Since there's always one answer for your dataset version:
    answer_char_start = example['answers']['answer_start'][0]
    answer_text = example['answers']['text'][0]
    answer_char_end = answer_char_start + len(answer_text)

    token_answer_start_idx = -1
    token_answer_end_idx = -1

    # Find token start index
    # Iterate through tokens and their character offsets
    for i, token in enumerate(context_doc):
        # Check if the answer's character start falls within the current token's span
        if token.idx <= answer_char_start < token.idx + len(token.text):
            token_answer_start_idx = i
            break
    # If not found by the above (e.g., answer starts exactly at a token boundary after whitespace)
    # try finding the first token whose start offset is >= answer_char_start
    if token_answer_start_idx == -1:
        for i, token in enumerate(context_doc):
            if token.idx >= answer_char_start:
                # Check if this token or the previous one is a better fit
                # This might need more refinement if answers can be leading/trailing spaces of tokens
                if i > 0 and (context_doc[i-1].idx + len(context_doc[i-1].text) > answer_char_start):
                     # This condition suggests the previous token might be part of the start
                     # This area can be complex if tokenization and answer boundaries are tricky
                     # For now, we'll stick to simpler logic often sufficient for SQuAD
                     pass # For now, we assume the first token whose .idx >= answer_char_start is fine if the previous check failed.
                token_answer_start_idx = i
                break


    # Find token end index
    if token_answer_start_idx != -1:
        for i in range(token_answer_start_idx, len(context_doc)):
            token = context_doc[i]
            # The end token is the last token whose text is part of the answer.
            # So, if this token's end character position is >= the answer's character end, it's a candidate.
            if (token.idx + len(token.text)) >= answer_char_end:
                token_answer_end_idx = i
                break
        # Sanity check: if token_answer_end_idx is found, ensure the span isn't empty
        # and that the reconstructed text roughly matches. This is more for debugging.
        # if token_answer_start_idx != -1 and token_answer_end_idx != -1 :
        #     reconstructed_answer = "".join(t.text_with_ws for t in context_doc[token_answer_start_idx:token_answer_end_idx+1]).strip()
        #     if not answer_text in reconstructed_answer : # Be careful with exact match due to tokenization nuances
        #         print(f"Warning: Mismatch for example ID {example['id']}. Original: '{answer_text}', Reconstructed: '{reconstructed_answer}' from tokens {token_answer_start_idx}-{token_answer_end_idx}")


    # Fallback or error handling if spans are not found properly
    if token_answer_start_idx == -1 or token_answer_end_idx == -1:
        # This indicates an issue with answer span mapping.
        # For SQuAD v1.1, an answer is guaranteed.
        # You might need to refine the logic or flag these examples.
        print(f"Warning: Could not map answer for example ID {example['id']}. Context: '{example['context']}', Answer: '{answer_text}' at char {answer_char_start}. Tok_start: {token_answer_start_idx}, Tok_end: {token_answer_end_idx}")
        # For BiDAF, you need valid start/end token indices.
        # Assigning -1 or skipping might be options, but ideally, all SQuAD v1.1 answers should be mappable.

    return {
        "tokenized_context": tokenized_context,
        "tokenized_question": tokenized_question,
        "context_token_char_offsets": context_token_char_offsets, # Good for debugging
        "token_answer_start": token_answer_start_idx,
        "token_answer_end": token_answer_end_idx,
        "id": example['id'] # Keep id for debugging
    }

# Then process the whole dataset:
print("\nMapping the tokenization function to the training dataset...")
squad_train_tokenized = squad_train.map(tokenize_example_spacy, batched=False)
print("Mapping the tokenization function to the validation dataset...")
squad_validation_tokenized = squad_validation.map(tokenize_example_spacy, batched=False)

# Verify the new columns and an example
print("\nFeatures of tokenized training data:", squad_train_tokenized.features)
print("\nFirst example of tokenized training data:")
print(squad_train_tokenized[0])


Mapping the tokenization function to the training dataset...


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Mapping the tokenization function to the validation dataset...


Map:   0%|          | 0/10570 [00:00<?, ? examples/s]


Features of tokenized training data: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None), 'tokenized_context': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'tokenized_question': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'context_token_char_offsets': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_answer_start': Value(dtype='int64', id=None), 'token_answer_end': Value(dtype='int64', id=None)}

First example of tokenized training data:
{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in 

In [34]:
squad_train_tokenized.features

{'id': Value(dtype='string', id=None),
 'title': Value(dtype='string', id=None),
 'context': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None),
 'tokenized_context': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'tokenized_question': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'context_token_char_offsets': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'token_answer_start': Value(dtype='int64', id=None),
 'token_answer_end': Value(dtype='int64', id=None)}

In [36]:
squad_train_tokenized['tokenized_context'][0]

['Architecturally',
 ',',
 'the',
 'school',
 'has',
 'a',
 'Catholic',
 'character',
 '.',
 'Atop',
 'the',
 'Main',
 'Building',
 "'s",
 'gold',
 'dome',
 'is',
 'a',
 'golden',
 'statue',
 'of',
 'the',
 'Virgin',
 'Mary',
 '.',
 'Immediately',
 'in',
 'front',
 'of',
 'the',
 'Main',
 'Building',
 'and',
 'facing',
 'it',
 ',',
 'is',
 'a',
 'copper',
 'statue',
 'of',
 'Christ',
 'with',
 'arms',
 'upraised',
 'with',
 'the',
 'legend',
 '"',
 'Venite',
 'Ad',
 'Me',
 'Omnes',
 '"',
 '.',
 'Next',
 'to',
 'the',
 'Main',
 'Building',
 'is',
 'the',
 'Basilica',
 'of',
 'the',
 'Sacred',
 'Heart',
 '.',
 'Immediately',
 'behind',
 'the',
 'basilica',
 'is',
 'the',
 'Grotto',
 ',',
 'a',
 'Marian',
 'place',
 'of',
 'prayer',
 'and',
 'reflection',
 '.',
 'It',
 'is',
 'a',
 'replica',
 'of',
 'the',
 'grotto',
 'at',
 'Lourdes',
 ',',
 'France',
 'where',
 'the',
 'Virgin',
 'Mary',
 'reputedly',
 'appeared',
 'to',
 'Saint',
 'Bernadette',
 'Soubirous',
 'in',
 '1858',
 '.',
 'At

In [38]:
t = [
    np.max([len(x) for x in ctx]) for ctx in squad_train_tokenized['tokenized_context']
]
np.quantile(t, [0.25, 0.5, 0.75, 0.9, 0.95, 0.99])

array([12., 13., 14., 15., 16., 20.])

In [39]:
t = np.array(t)

In [53]:
np.argmax(t)

np.int64(72926)

In [77]:
squad_train_tokenized['tokenized_context'][np.argmax(t == 28)]

['GE',
 "'s",
 'history',
 'of',
 'working',
 'with',
 'turbines',
 'in',
 'the',
 'power',
 '-',
 'generation',
 'field',
 'gave',
 'them',
 'the',
 'engineering',
 'know',
 '-',
 'how',
 'to',
 'move',
 'into',
 'the',
 'new',
 'field',
 'of',
 'aircraft',
 'turbosuperchargers.[citation',
 'needed',
 ']',
 'Led',
 'by',
 'Sanford',
 'Alexander',
 'Moss',
 ',',
 'GE',
 'introduced',
 'the',
 'first',
 'superchargers',
 'during',
 'World',
 'War',
 'I',
 ',',
 'and',
 'continued',
 'to',
 'develop',
 'them',
 'during',
 'the',
 'Interwar',
 'period',
 '.',
 'Superchargers',
 'became',
 'indispensable',
 'in',
 'the',
 'years',
 'immediately',
 'prior',
 'to',
 'World',
 'War',
 'II',
 ',',
 'and',
 'GE',
 'was',
 'the',
 'world',
 'leader',
 'in',
 'exhaust',
 '-',
 'driven',
 'supercharging',
 'when',
 'the',
 'war',
 'started',
 '.',
 'This',
 'experience',
 ',',
 'in',
 'turn',
 ',',
 'made',
 'GE',
 'a',
 'natural',
 'selection',
 'to',
 'develop',
 'the',
 'Whittle',
 'W.1',
 'jet

In [76]:
np.sum(t == 28)

np.int64(15)

In [19]:
import os # For checking if files exist

# --- Configuration for saving/loading ---
# Define paths where you want to save your tokenized datasets
tokenized_train_path = "./squad_train_tokenized_spacy"
tokenized_validation_path = "./squad_validation_tokenized_spacy"

# Save the tokenized datasets
print("\nSaving tokenized training data to disk...")
squad_train_tokenized.save_to_disk(tokenized_train_path)
print(f"Saved tokenized training data to {tokenized_train_path}")

print("\nSaving tokenized validation data to disk...")
squad_validation_tokenized.save_to_disk(tokenized_validation_path)
print(f"Saved tokenized validation data to {tokenized_validation_path}")
print("Successfully tokenized and saved data.")


Saving tokenized training data to disk...


Saving the dataset (0/1 shards):   0%|          | 0/87599 [00:00<?, ? examples/s]

Saved tokenized training data to ./squad_train_tokenized_spacy

Saving tokenized validation data to disk...


Saving the dataset (0/1 shards):   0%|          | 0/10570 [00:00<?, ? examples/s]

Saved tokenized validation data to ./squad_validation_tokenized_spacy
Successfully tokenized and saved data.


In [20]:
from collections import Counter

# Assuming squad_train_tokenized is loaded and available
# squad_train_tokenized = Dataset.load_from_disk("./squad_train_tokenized_spacy")
# squad_validation_tokenized = Dataset.load_from_disk("./squad_validation_tokenized_spacy")


# --- Configuration for Vocabulary ---
MIN_WORD_FREQ = 1  # Minimum frequency for a word to be included in the vocabulary
VOCAB_SIZE_LIMIT = None # Set to an integer if you want to cap vocab size, e.g., 50000

# Special tokens
PAD_TOKEN = "<PAD>"
UNK_TOKEN = "<UNK>"
# You can add more special tokens if needed, e.g., SOS_TOKEN, EOS_TOKEN

special_tokens = [PAD_TOKEN, UNK_TOKEN]

# --- 1. Count word frequencies from the training data ---
print("Building vocabulary...")
word_counter = Counter()

# Iterate through tokenized contexts and questions in the training set
for example in squad_train_tokenized:
    word_counter.update(example['tokenized_context'])
    word_counter.update(example['tokenized_question'])

print(f"Found {len(word_counter)} unique words in the training data.")

# --- 2. Create word_to_idx and idx_to_word mappings ---
# Start with special tokens
word_to_idx = {token: idx for idx, token in enumerate(special_tokens)}
idx_to_word = {idx: token for token, idx in word_to_idx.items()}

# Add words from the counter that meet the minimum frequency
# Sort words by frequency (most common first) for potential vocab size limiting
sorted_words = word_counter.most_common()

for word, count in sorted_words:
    if count >= MIN_WORD_FREQ:
        if word not in word_to_idx: # Ensure not to overwrite special tokens if they appeared in data
            idx = len(word_to_idx)
            word_to_idx[word] = idx
            idx_to_word[idx] = word
    if VOCAB_SIZE_LIMIT and len(word_to_idx) >= VOCAB_SIZE_LIMIT:
        print(f"Reached vocabulary size limit of {VOCAB_SIZE_LIMIT}.")
        break

print(f"Vocabulary size (including special tokens): {len(word_to_idx)}")
print(f"First 10 words in vocab: { {k: word_to_idx[k] for k in list(word_to_idx)[:10]} }")
print(f"Example: ID for '{UNK_TOKEN}' is {word_to_idx[UNK_TOKEN]}")
if 'the' in word_to_idx: # Check a common word
    print(f"Example: ID for 'the' is {word_to_idx['the']}")


# --- (Optional) Save the vocabulary ---
# You might want to save your word_to_idx and idx_to_word
# For example, using pickle or json
import json

vocab_path = "./squad_vocab_spacy.json"
print(f"Saving vocabulary to {vocab_path}...")
with open(vocab_path, 'w') as f:
    json.dump({'word_to_idx': word_to_idx, 'idx_to_word': idx_to_word}, f)
print("Vocabulary saved.")

# To load it back later:
# with open(vocab_path, 'r') as f:
#     vocab_data = json.load(f)
#     word_to_idx = vocab_data['word_to_idx']
#     idx_to_word = vocab_data['idx_to_word']
# print("Vocabulary loaded.")

Building vocabulary...
Found 103961 unique words in the training data.
Vocabulary size (including special tokens): 103963
First 10 words in vocab: {'<PAD>': 0, '<UNK>': 1, 'the': 2, ',': 3, 'of': 4, '.': 5, 'and': 6, 'in': 7, 'to': 8, 'a': 9}
Example: ID for '<UNK>' is 1
Example: ID for 'the' is 2
Saving vocabulary to ./squad_vocab_spacy.json...
Vocabulary saved.


In [26]:
import numpy as np
import torch # We'll need this for the final embedding matrix
import json # For loading the vocabulary if you saved it

# --- Configuration ---
# Path to your downloaded GloVe file (e.g., glove.6B.100d.txt)
GLOVE_FILE_PATH = "./glove.840B.300d.txt" #  <-- Make sure this path is correct!
EMBEDDING_DIM = 300 #  <-- This must match the GloVe file you are using (e.g., 100d, 300d)
VOCAB_PATH = "./squad_vocab_spacy.json" # Path where you saved your vocabulary

# Special tokens
PAD_TOKEN = "<PAD>"
UNK_TOKEN = "<UNK>"

# --- 1. Load your vocabulary ---
print(f"Loading vocabulary from {VOCAB_PATH}...")
try:
    with open(VOCAB_PATH, 'r') as f:
        vocab_data = json.load(f)
        word_to_idx = vocab_data['word_to_idx']
        idx_to_word = vocab_data['idx_to_word'] # Not strictly needed here, but good to have
    print("Vocabulary loaded successfully.")
    VOCAB_SIZE = len(word_to_idx)
    print(f"Vocabulary size: {VOCAB_SIZE}")
except FileNotFoundError:
    print(f"Error: Vocabulary file not found at {VOCAB_PATH}. Please ensure you've built and saved the vocabulary first.")
    exit() # Or handle this error appropriately

# --- 2. Load GloVe vectors from file ---
print(f"Loading GloVe vectors from {GLOVE_FILE_PATH}...")
glove_vectors = {}
try:
    with open(GLOVE_FILE_PATH, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            parts = line.strip().split()
            word = parts[0]
            try:
                vector = np.asarray(parts[1:], dtype='float32')
            except ValueError as e:
                continue # Skip lines with malformed vectors
            glove_vectors[word] = vector
    print(f"Loaded {len(glove_vectors)} word vectors from GloVe.")
except FileNotFoundError:
    print(f"Error: GloVe file not found at {GLOVE_FILE_PATH}. Please download it and check the path.")
    exit() # Or handle appropriately

# --- 3. Create the embedding matrix ---
print("Creating embedding matrix...")
# Initialize with random values (e.g., from a normal distribution) or zeros
# Small random values are often better than zeros for non-PAD, non-GloVe words.
# We will specifically handle PAD and UNK.
embedding_matrix = np.random.normal(scale=0.6, size=(VOCAB_SIZE, EMBEDDING_DIM)).astype(np.float32)
# Alternatively, initialize with zeros:
# embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM), dtype=np.float32)


hits = 0
misses = 0

for word, idx in word_to_idx.items():
    if word == PAD_TOKEN:
        # Ensure PAD token vector is all zeros
        embedding_matrix[idx] = np.zeros(EMBEDDING_DIM, dtype=np.float32)
        hits +=1 # Considered a "hit" as we intentionally set it
    elif word in glove_vectors:
        embedding_matrix[idx] = glove_vectors[word]
        hits += 1
    else:
        # Word not in GloVe (or it's the UNK token itself if not in GloVe)
        # It will retain its random initialization from above.
        # If you initialized with zeros, you might want to assign a specific random vector for UNK here:
        # if word == UNK_TOKEN:
        #    embedding_matrix[idx] = np.random.normal(scale=0.6, size=(EMBEDDING_DIM, )).astype(np.float32)
        misses += 1

print(f"Embedding matrix shape: {embedding_matrix.shape}")
print(f"Words found in GloVe (hits): {hits}")
print(f"Words not found in GloVe (misses): {misses} (these will use random initialization or UNK vector)")

# Specifically check the UNK token's embedding (it should be the random init if not in GloVe)
if UNK_TOKEN in word_to_idx:
    unk_idx = word_to_idx[UNK_TOKEN]
    # If UNK_TOKEN was not in GloVe, its vector will be the initially random one.
    # If it happened to be in GloVe (unlikely for "<UNK>"), it would have the GloVe vector.
    print(f"Vector for UNK token (index {unk_idx}): {embedding_matrix[unk_idx][:10]}...") # Print first 10 dims
else:
    print("Warning: UNK_TOKEN not found in word_to_idx. This is unexpected.")

if PAD_TOKEN in word_to_idx:
     pad_idx = word_to_idx[PAD_TOKEN]
     print(f"Vector for PAD token (index {pad_idx}): {embedding_matrix[pad_idx][:10]}...")
else:
    print("Warning: PAD_TOKEN not found in word_to_idx. This is unexpected.")


# --- 4. Convert to PyTorch Tensor ---
embedding_matrix_tensor = torch.tensor(embedding_matrix, dtype=torch.float32)
print(f"Embedding matrix converted to PyTorch tensor with shape: {embedding_matrix_tensor.shape}")

# --- (Optional) Save the embedding matrix tensor ---
embedding_matrix_path = "./squad_embedding_matrix_spacy.pt"
print(f"Saving embedding matrix tensor to {embedding_matrix_path}...")
torch.save(embedding_matrix_tensor, embedding_matrix_path)
print("Embedding matrix tensor saved.")

# To load it back later:
# embedding_matrix_tensor = torch.load(embedding_matrix_path)
# print("Embedding matrix tensor loaded.")

Loading vocabulary from ./squad_vocab_spacy.json...
Vocabulary loaded successfully.
Vocabulary size: 103963
Loading GloVe vectors from ./glove.840B.300d.txt...
Loaded 2195884 word vectors from GloVe.
Creating embedding matrix...
Embedding matrix shape: (103963, 300)
Words found in GloVe (hits): 86556
Words not found in GloVe (misses): 17407 (these will use random initialization or UNK vector)
Vector for UNK token (index 1): [-0.18693003  0.59416074 -0.40828347  0.6995341  -0.15922594  0.29638168
  0.30654564 -0.00461517 -0.2958318   0.5320532 ]...
Vector for PAD token (index 0): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]...
Embedding matrix converted to PyTorch tensor with shape: torch.Size([103963, 300])
Saving embedding matrix tensor to ./squad_embedding_matrix_spacy.pt...
Embedding matrix tensor saved.


In [27]:
import numpy as np
import torch
import json
from datasets import Dataset # Assuming you're using Hugging Face datasets

# --- Configuration ---
VOCAB_PATH = "./squad_vocab_spacy.json" # Path where you saved your vocabulary
PAD_TOKEN = "<PAD>"
UNK_TOKEN = "<UNK>"

# Placeholder maximum lengths -  ADJUST THESE BASED ON YOUR DATA ANALYSIS
MAX_CONTEXT_LEN = 512 # Common for SQuAD, but verify
MAX_QUESTION_LEN = 60  # Common for SQuAD, but verify

# --- 1. Load your vocabulary ---
print(f"Loading vocabulary from {VOCAB_PATH}...")
try:
    with open(VOCAB_PATH, 'r') as f:
        vocab_data = json.load(f)
        word_to_idx = vocab_data['word_to_idx']
    print("Vocabulary loaded successfully.")
    PAD_ID = word_to_idx[PAD_TOKEN]
    UNK_ID = word_to_idx[UNK_TOKEN]
except FileNotFoundError:
    print(f"Error: Vocabulary file not found at {VOCAB_PATH}. Please ensure you've built and saved the vocabulary first.")
    exit()
except KeyError as e:
    print(f"Error: Special token {e} not found in vocabulary. Please ensure PAD_TOKEN and UNK_TOKEN are in your vocab.")
    exit()

# --- 2. Load your tokenized datasets (if not already in memory) ---
# Replace with your actual loading logic if needed
# For demonstration, I'll assume they are loaded as `squad_train_tokenized` and `squad_validation_tokenized`
# squad_train_tokenized = Dataset.load_from_disk("./squad_train_tokenized_spacy")
# squad_validation_tokenized = Dataset.load_from_disk("./squad_validation_tokenized_spacy")
# Make sure these datasets have 'tokenized_context' and 'tokenized_question' columns.

# --- 3. Define the processing function ---
def numericalize_and_pad(example):
    # Numericalize context
    context_tokens = example['tokenized_context']
    context_ids = [word_to_idx.get(token, UNK_ID) for token in context_tokens]
    original_context_len = len(context_ids)

    # Numericalize question
    question_tokens = example['tokenized_question']
    question_ids = [word_to_idx.get(token, UNK_ID) for token in question_tokens]
    original_question_len = len(question_ids)

    # Pad/Truncate context
    if len(context_ids) > MAX_CONTEXT_LEN:
        padded_context_ids = context_ids[:MAX_CONTEXT_LEN]
    else:
        padded_context_ids = context_ids + [PAD_ID] * (MAX_CONTEXT_LEN - len(context_ids))

    # Pad/Truncate question
    if len(question_ids) > MAX_QUESTION_LEN:
        padded_question_ids = question_ids[:MAX_QUESTION_LEN]
    else:
        padded_question_ids = question_ids + [PAD_ID] * (MAX_QUESTION_LEN - len(question_ids))

    return {
        'context_ids': padded_context_ids,
        'question_ids': padded_question_ids,
        'original_context_len': original_context_len, # Store original length
        'original_question_len': original_question_len, # Store original length
        # Keep answer token indices as they are already token-level
        'token_answer_start': example['token_answer_start'],
        'token_answer_end': example['token_answer_end'],
        'id': example['id'] # Keep id for debugging
    }

# --- 4. Apply the function to your datasets ---
# This assumes squad_train_tokenized and squad_validation_tokenized are loaded
# and are Hugging Face Dataset objects.

# Dummy datasets for demonstration if you don't have them loaded yet:
# This is just to make the script runnable for illustration.
# Replace with your actual dataset loading.
if 'squad_train_tokenized' not in locals():
    print("Creating dummy tokenized datasets for demonstration...")
    dummy_data = {
        'id': ['1', '2'],
        'tokenized_context': [['This', 'is', 'a', 'context', '.'], ['Another', 'example', 'context', 'here', '.']],
        'tokenized_question': [['What', 'is', 'this', '?'], ['Example', 'question', '.']],
        'token_answer_start': [2, 1],
        'token_answer_end': [3, 2]
    }
    squad_train_tokenized = Dataset.from_dict(dummy_data)
    squad_validation_tokenized = Dataset.from_dict(dummy_data)
    print("Dummy datasets created.")


print("Numericalizing and padding training data...")
squad_train_processed = squad_train_tokenized.map(numericalize_and_pad, batched=False)
print("Numericalizing and padding validation data...")
squad_validation_processed = squad_validation_tokenized.map(numericalize_and_pad, batched=False)

# --- 5. Verify the output ---
print("\nProcessed training data features:", squad_train_processed.features)
if len(squad_train_processed) > 0:
    print("\nFirst example of processed training data:")
    ex = squad_train_processed[0]
    print(f"  Context IDs (first 20): {ex['context_ids'][:20]}...")
    print(f"  Original Context Length: {ex['original_context_len']}")
    print(f"  Padded Context Length: {len(ex['context_ids'])}")
    print(f"  Question IDs (first 20): {ex['question_ids'][:20]}...")
    print(f"  Original Question Length: {ex['original_question_len']}")
    print(f"  Padded Question Length: {len(ex['question_ids'])}")
    print(f"  Answer Start Token: {ex['token_answer_start']}")
    print(f"  Answer End Token: {ex['token_answer_end']}")

# --- (Optional) Save the processed datasets ---
processed_train_path = "./squad_train_processed_spacy.hf"
processed_validation_path = "./squad_validation_processed_spacy.hf"

print(f"\nSaving processed training data to {processed_train_path}...")
squad_train_processed.save_to_disk(processed_train_path)
print("Processed training data saved.")

print(f"Saving processed validation data to {processed_validation_path}...")
squad_validation_processed.save_to_disk(processed_validation_path)
print("Processed validation data saved.")

# To load them back:
# squad_train_processed = Dataset.load_from_disk(processed_train_path)
# squad_validation_processed = Dataset.load_from_disk(processed_validation_path)

Loading vocabulary from ./squad_vocab_spacy.json...
Vocabulary loaded successfully.
Numericalizing and padding training data...


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Numericalizing and padding validation data...


Map:   0%|          | 0/10570 [00:00<?, ? examples/s]


Processed training data features: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None), 'tokenized_context': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'tokenized_question': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'context_token_char_offsets': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_answer_start': Value(dtype='int64', id=None), 'token_answer_end': Value(dtype='int64', id=None), 'context_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'question_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'original_context_len': Value(dtype='int64', id=None), 'original_question_len': Value(dtype='int64', id=N

Saving the dataset (0/2 shards):   0%|          | 0/87599 [00:00<?, ? examples/s]

Processed training data saved.
Saving processed validation data to ./squad_validation_processed_spacy.hf...


Saving the dataset (0/1 shards):   0%|          | 0/10570 [00:00<?, ? examples/s]

Processed validation data saved.


In [78]:
from collections import Counter
import json
from datasets import Dataset # Assuming you're using Hugging Face datasets

# --- Configuration for Character Vocabulary ---
CHAR_VOCAB_PATH = "./squad_char_vocab.json"
CHAR_PAD_TOKEN = "<CHAR_PAD>" # Padding character for words shorter than MAX_WORD_LEN
CHAR_UNK_TOKEN = "<CHAR_UNK>" # For unknown characters (less common if vocab built from all data)
# MAX_WORD_LEN is defined later, user chose 25

# --- Load previously tokenized data (needed to extract words for char vocab) ---
# This assumes 'squad_train_tokenized' has 'tokenized_context' and 'tokenized_question'
# For demonstration, if not loaded, use a dummy. Replace with your actual loading.
# tokenized_train_path = "./squad_train_tokenized_spacy"
# try:
#     squad_train_tokenized_for_char_vocab = Dataset.load_from_disk(tokenized_train_path)
#     print(f"Loaded tokenized training data from {tokenized_train_path} for char vocab building.")
# except FileNotFoundError:
#     print(f"Warning: Tokenized training data not found at {tokenized_train_path}. Using dummy data for char vocab.")
#     squad_train_tokenized_for_char_vocab = Dataset.from_dict({
#         'tokenized_context': [['Hello', 'world'], ['Test']],
#         'tokenized_question': [['How', 'are', 'you', '?'], ['Why', '?']]
#     })

def build_char_vocab(tokenized_dataset, special_tokens):
    print("Building character vocabulary...")
    char_counter = Counter()
    for example in tokenized_dataset:
        for word in example['tokenized_context']:
            char_counter.update(word)
        for word in example['tokenized_question']:
            char_counter.update(word)

    char_to_idx = {token: idx for idx, token in enumerate(special_tokens)}
    idx_to_char = {idx: token for token, idx in char_to_idx.items()}

    for char, count in char_counter.most_common():
        if char not in char_to_idx:
            idx = len(char_to_idx)
            char_to_idx[char] = idx
            idx_to_char[idx] = char

    print(f"Character vocabulary size (including special tokens): {len(char_to_idx)}")
    return char_to_idx, idx_to_char

# --- Main script execution for char vocab ---
# Assuming squad_train_tokenized is available (e.g., loaded from disk)
# Replace this with your actual loading of squad_train_tokenized if needed:
# squad_train_tokenized = Dataset.load_from_disk("./squad_train_tokenized_spacy")

# For demonstration, let's ensure a dummy `squad_train_tokenized` exists if not loaded:
if 'squad_train_tokenized' not in locals():
    print("Creating dummy 'squad_train_tokenized' for char vocab building demonstration.")
    dummy_data_for_char_vocab = {
        'id': ['1', '2'],
        'tokenized_context': [['This', 'is', 'a', 'context', 'example-wordlong'], ['Another', 'example', '.']],
        'tokenized_question': [['What', 'is', 'this', '?'], ['Example', 'question', '.']],
        'token_answer_start': [2, 1],
        'token_answer_end': [3, 2]
    }
    squad_train_tokenized = Dataset.from_dict(dummy_data_for_char_vocab)


char_special_tokens = [CHAR_PAD_TOKEN, CHAR_UNK_TOKEN]
char_to_idx, idx_to_char = build_char_vocab(squad_train_tokenized, char_special_tokens)

print(f"First 10 chars in vocab: { {k: char_to_idx[k] for k in list(char_to_idx)[:min(10, len(char_to_idx))]} }")
CHAR_PAD_ID = char_to_idx[CHAR_PAD_TOKEN]
CHAR_UNK_ID = char_to_idx[CHAR_UNK_TOKEN]
print(f"Character PAD ID: {CHAR_PAD_ID}")
print(f"Character UNK ID: {CHAR_UNK_ID}")


print(f"Saving character vocabulary to {CHAR_VOCAB_PATH}...")
with open(CHAR_VOCAB_PATH, 'w') as f:
    json.dump({'char_to_idx': char_to_idx, 'idx_to_char': idx_to_char}, f)
print("Character vocabulary saved.")

# For later use:
# with open(CHAR_VOCAB_PATH, 'r') as f:
#     char_vocab_data = json.load(f)
#     char_to_idx = char_vocab_data['char_to_idx']
#     idx_to_char = char_vocab_data['idx_to_char']
#     CHAR_PAD_ID = char_to_idx[CHAR_PAD_TOKEN]
#     CHAR_UNK_ID = char_to_idx[CHAR_UNK_TOKEN]

Building character vocabulary...
Character vocabulary size (including special tokens): 1374
First 10 chars in vocab: {'<CHAR_PAD>': 0, '<CHAR_UNK>': 1, 'e': 2, 't': 3, 'a': 4, 'i': 5, 'n': 6, 'o': 7, 'r': 8, 's': 9}
Character PAD ID: 0
Character UNK ID: 1
Saving character vocabulary to ./squad_char_vocab.json...
Character vocabulary saved.


In [5]:
import numpy as np
import torch # Not strictly needed here, but often imported together
from datasets import Dataset # Already imported
import json

CHAR_VOCAB_PATH = "./squad_char_vocab.json"
CHAR_PAD_TOKEN = "<CHAR_PAD>" # Padding character for words shorter than MAX_WORD_LEN
CHAR_UNK_TOKEN = "<CHAR_UNK>" # For unknown characters (less common if vocab built from all data)

# --- Configuration & Hyperparameters ---
# Paths (ensure these match your saved files)
PROCESSED_TRAIN_PATH = "./squad_train_processed_char.hf"
PROCESSED_VALIDATION_PATH = "./squad_validation_processed_char.hf"
WORD_VOCAB_PATH = "./squad_vocab_spacy.json"
CHAR_VOCAB_PATH = "./squad_char_vocab.json"
PRETRAINED_WORD_EMB_PATH = "./squad_embedding_matrix_spacy.pt" # GloVe matrix

# Model & Embedding Hyperparameters (should match your model definition)
# Word Embeddings
WORD_EMBEDDING_DIM = 300 # User specified
# Character Embeddings (based on BiDAF paper / common practice)
CHAR_EMBEDDING_DIM = 8    # Dimension of individual char embedding
CHAR_CNN_OUT_CHANNELS = 100 # Output dim of char CNN, effectively char-level word emb dim
CHAR_CNN_KERNEL_SIZE = 5
# Model Structure
HIDDEN_SIZE = 300 # 'd' in BiDAF paper, often same as word_embedding_dim
NUM_HIGHWAY_LAYERS = 2
DROPOUT_RATE = 0.2
# MAX_WORD_LEN = 25 # Defined during preprocessing

# Training Hyperparameters
LEARNING_RATE = 0.001
BATCH_SIZE = 10 # Adjust based on your GPU memory
NUM_EPOCHS = 10 # Start with a few, increase later
CLIP_GRAD_NORM = 5.0 # For gradient clipping

squad_train_tokenized = Dataset.load_from_disk("./squad_train_tokenized_spacy")
squad_validation_tokenized = Dataset.load_from_disk("./squad_validation_tokenized_spacy") # or however you load it

# --- Configuration from previous steps (ensure these are loaded/defined) ---
WORD_VOCAB_PATH = "./squad_vocab_spacy.json" # Path to your word vocabulary
# CHAR_VOCAB_PATH is defined above
WORD_PAD_TOKEN = "<PAD>"
WORD_UNK_TOKEN = "<UNK>"

MAX_CONTEXT_LEN = 512  # User specified
MAX_QUESTION_LEN = 60 # User specified
MAX_WORD_LEN = 25      # User specified

# --- Load Word Vocabulary ---
print(f"Loading word vocabulary from {WORD_VOCAB_PATH}...")
try:
    with open(WORD_VOCAB_PATH, 'r') as f:
        word_vocab_data = json.load(f)
        word_to_idx = word_vocab_data['word_to_idx']
    WORD_PAD_ID = word_to_idx[WORD_PAD_TOKEN]
    WORD_UNK_ID = word_to_idx[WORD_UNK_TOKEN]
    print("Word vocabulary loaded.")
except FileNotFoundError:
    print(f"Error: Word vocabulary file not found at {WORD_VOCAB_PATH}.")
    exit()
except KeyError as e:
    print(f"Error: Word special token {e} not found in word vocabulary.")
    exit()

# --- Load Character Vocabulary (built in Part 1 or loaded if exists) ---
print(f"Loading character vocabulary from {CHAR_VOCAB_PATH}...")
try:
    with open(CHAR_VOCAB_PATH, 'r') as f:
        char_vocab_data = json.load(f)
        char_to_idx = char_vocab_data['char_to_idx']
    CHAR_PAD_ID = char_to_idx[CHAR_PAD_TOKEN]
    CHAR_UNK_ID = char_to_idx[CHAR_UNK_TOKEN] # Ensure this was created if needed
    print("Character vocabulary loaded.")
except FileNotFoundError:
    print(f"Error: Character vocabulary file not found at {CHAR_VOCAB_PATH}.")
    exit()
except KeyError as e:
    print(f"Error: Character special token {e} not found in character vocabulary.")
    exit()


def process_tokens_for_model(tokens, max_tokens_len, word_to_idx_local, char_to_idx_local,
                             max_word_len_local, word_pad_id_local, word_unk_id_local,
                             char_pad_id_local, char_unk_id_local):
    # 1. Word ID processing
    word_ids = [word_to_idx_local.get(token, word_unk_id_local) for token in tokens]
    original_len = len(word_ids) # True length of word tokens

    # Pad/Truncate word IDs
    if len(word_ids) > max_tokens_len:
        padded_word_ids = word_ids[:max_tokens_len]
    else:
        padded_word_ids = word_ids + [word_pad_id_local] * (max_tokens_len - len(word_ids))

    # 2. Character ID processing
    char_ids_for_words = []
    # Iterate up to the potentially truncated number of words (max_tokens_len)
    # but use original_len to access actual tokens
    num_words_to_process_chars_for = min(original_len, max_tokens_len)

    for i in range(max_tokens_len):
        if i < num_words_to_process_chars_for: # If it's an actual word within the (potentially truncated) sequence
            word = tokens[i] # Access from original tokens list
            current_word_char_ids = [char_to_idx_local.get(char, char_unk_id_local) for char in word]
            if len(current_word_char_ids) > max_word_len_local:
                padded_word_char_ids = current_word_char_ids[:max_word_len_local]
            else:
                padded_word_char_ids = current_word_char_ids + [char_pad_id_local] * (max_word_len_local - len(current_word_char_ids))
        else: # It's a padding word slot (either original padding or due to context truncation)
            padded_word_char_ids = [char_pad_id_local] * max_word_len_local
        char_ids_for_words.append(padded_word_char_ids)

    return padded_word_ids, char_ids_for_words, original_len


def full_preprocess_function(example):
    # Process context
    context_word_ids, context_char_ids, original_context_len = process_tokens_for_model(
        tokens=example['tokenized_context'],
        max_tokens_len=MAX_CONTEXT_LEN,
        word_to_idx_local=word_to_idx, # Make sure word_to_idx is accessible
        char_to_idx_local=char_to_idx,   # Make sure char_to_idx is accessible
        max_word_len_local=MAX_WORD_LEN,
        word_pad_id_local=WORD_PAD_ID, # Make sure WORD_PAD_ID is accessible
        word_unk_id_local=WORD_UNK_ID, # Make sure WORD_UNK_ID is accessible
        char_pad_id_local=CHAR_PAD_ID, # Make sure CHAR_PAD_ID is accessible
        char_unk_id_local=CHAR_UNK_ID  # Make sure CHAR_UNK_ID is accessible
    )

    # Process question
    question_word_ids, question_char_ids, original_question_len = process_tokens_for_model(
        tokens=example['tokenized_question'],
        max_tokens_len=MAX_QUESTION_LEN,
        word_to_idx_local=word_to_idx,
        char_to_idx_local=char_to_idx,
        max_word_len_local=MAX_WORD_LEN,
        word_pad_id_local=WORD_PAD_ID,
        word_unk_id_local=WORD_UNK_ID,
        char_pad_id_local=CHAR_PAD_ID,
        char_unk_id_local=CHAR_UNK_ID
    )

    # Get original answer token indices from the example
    # These were computed based on the original (untruncated) tokenized_context
    ans_start_orig = example['token_answer_start']
    ans_end_orig = example['token_answer_end']

    final_ans_start = ans_start_orig
    final_ans_end = ans_end_orig

    # Check if the original answer indices are valid *after* context truncation to MAX_CONTEXT_LEN
    # The logits produced by the model will have a dimension of MAX_CONTEXT_LEN.
    # So, target indices must be < MAX_CONTEXT_LEN.
    if ans_start_orig != -1 : # Check if it was a validly mapped answer initially
        if ans_start_orig >= MAX_CONTEXT_LEN or ans_end_orig >= MAX_CONTEXT_LEN:
            # If the start or end of the answer falls outside the truncated context length
            print(f"WARNING: Example {example['id']}: Original answer span ({ans_start_orig}, {ans_end_orig}) "
                  f"is partially or fully outside MAX_CONTEXT_LEN ({MAX_CONTEXT_LEN}). "
                  f"Marking this example's answer to be ignored by loss function.")
            final_ans_start = -1  # To be ignored by CrossEntropyLoss(ignore_index=-1)
            final_ans_end = -1    # To be ignored by CrossEntropyLoss
        # Optional: a further sanity check if end somehow became less than start after some logic
        # Though with the above, if start is valid, end should be >= start unless original data was flawed
        # or original mapping set end < start for some reason.
        # if final_ans_start != -1 and final_ans_end < final_ans_start:
        #    print(f"WARNING: Example {example['id']}: Corrected ans_end ({final_ans_end}) < ans_start ({final_ans_start}). Setting to ignore.")
        #    final_ans_start = -1
        #    final_ans_end = -1

    # If ans_start_orig was already -1 (e.g., from an earlier "could not map" warning),
    # final_ans_start and final_ans_end will remain -1.

    return {
        'context_ids': context_word_ids,
        'context_char_ids': context_char_ids,
        'original_context_len': original_context_len,

        'question_ids': question_word_ids,
        'question_char_ids': question_char_ids,
        'original_question_len': original_question_len,

        'token_answer_start': final_ans_start, # Use the potentially adjusted values
        'token_answer_end': final_ans_end,   # Use the potentially adjusted values
        'id': example['id']
    }


# --- Apply to your datasets ---
# This assumes 'squad_train_tokenized' and 'squad_validation_tokenized' are loaded
# and are Hugging Face Dataset objects from the spaCy tokenization step.
# (Using the dummy `squad_train_tokenized` from Part 1 if no real one is loaded)

print("\nApplying full preprocessing (word & char) to training data...")
squad_train_processed_with_chars = squad_train_tokenized.map(full_preprocess_function, batched=False)
print("Applying full preprocessing (word & char) to validation data...")
# Ensure squad_validation_tokenized is loaded or defined similarly if you're running this standalone
if 'squad_validation_tokenized' not in locals():
    print("Creating dummy 'squad_validation_tokenized' for demonstration.")
    squad_validation_tokenized = Dataset.from_dict(squad_train_tokenized.to_dict()) # Just duplicate for now

squad_validation_processed_with_chars = squad_validation_tokenized.map(full_preprocess_function, batched=False)


# --- Verify the output ---
print("\nProcessed training data (with chars) features:", squad_train_processed_with_chars.features)
if len(squad_train_processed_with_chars) > 0:
    ex = squad_train_processed_with_chars[0]
    print("\nFirst example of processed training data (with chars):")
    print(f"  Context IDs (word level, first 10): {ex['context_ids'][:10]}...")
    print(f"  Context Char IDs (shape for first word): {np.array(ex['context_char_ids'][0]).shape if ex['context_char_ids'] else 'N/A'}")
    print(f"  Context Char IDs (first word, first 10 chars): {ex['context_char_ids'][0][:10] if ex['context_char_ids'] else 'N/A'}...")
    print(f"  Shape of context_char_ids field: {np.array(ex['context_char_ids']).shape}")
    print(f"  Original Context Length (words): {ex['original_context_len']}")
    print(f"  Padded Context Length (words): {len(ex['context_ids'])}")

    print(f"  Question IDs (word level, first 10): {ex['question_ids'][:10]}...")
    print(f"  Shape of question_char_ids field: {np.array(ex['question_char_ids']).shape}")
    print(f"  Original Question Length (words): {ex['original_question_len']}")

    print(f"  Answer Start Token: {ex['token_answer_start']}")
    print(f"  Answer End Token: {ex['token_answer_end']}")

# --- (Optional) Save the fully processed datasets ---
processed_train_char_path = "./squad_train_processed_char.hf"
processed_validation_char_path = "./squad_validation_processed_char.hf"

print(f"\nSaving fully processed training data to {processed_train_char_path}...")
squad_train_processed_with_chars.save_to_disk(processed_train_char_path)
print("Fully processed training data saved.")

print(f"Saving fully processed validation data to {processed_validation_char_path}...")
squad_validation_processed_with_chars.save_to_disk(processed_validation_char_path)
print("Fully processed validation data saved.")

Loading word vocabulary from ./squad_vocab_spacy.json...
Word vocabulary loaded.
Loading character vocabulary from ./squad_char_vocab.json...
Character vocabulary loaded.

Applying full preprocessing (word & char) to training data...


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Applying full preprocessing (word & char) to validation data...


Map:   0%|          | 0/10570 [00:00<?, ? examples/s]


Processed training data (with chars) features: {'id': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None), 'context': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None), 'tokenized_context': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'tokenized_question': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'context_token_char_offsets': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'token_answer_start': Value(dtype='int64', id=None), 'token_answer_end': Value(dtype='int64', id=None), 'context_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'context_char_ids': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None), 'original_context_len': Value(dtype='int64', id=No

Saving the dataset (0/22 shards):   0%|          | 0/87599 [00:00<?, ? examples/s]

Fully processed training data saved.
Saving fully processed validation data to ./squad_validation_processed_char.hf...


Saving the dataset (0/3 shards):   0%|          | 0/10570 [00:00<?, ? examples/s]

Fully processed validation data saved.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CharEmbedding(nn.Module):
    def __init__(self, char_vocab_size, char_embedding_dim, char_cnn_out_channels,
                 char_cnn_kernel_size, char_padding_idx, dropout_rate):
        super(CharEmbedding, self).__init__()

        self.char_embedding = nn.Embedding(
            num_embeddings=char_vocab_size,
            embedding_dim=char_embedding_dim,
            padding_idx=char_padding_idx
        )

        self.conv1d = nn.Conv1d(
            in_channels=char_embedding_dim,
            out_channels=char_cnn_out_channels,
            kernel_size=char_cnn_kernel_size,
            # Padding to maintain length for easier max-pooling across the sequence dimension later,
            # or use specific padding based on kernel size.
            # For kernel_size=5, padding=2 would keep length same if stride=1.
        )
        # The paper implies max-pooling over the resulting length of the convolution for each word.
        # If conv output length is L_out = L_in - kernel_size + 1 + 2*padding.
        # Let's use padding such that the output length is reasonable.
        # A common approach: padding = (kernel_size - 1) // 2 for 'same' style padding.

        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x_char_ids):
        # x_char_ids: (batch_size, seq_len, max_word_len)
        batch_size, seq_len, max_word_len = x_char_ids.shape

        # Reshape for embedding: (batch_size * seq_len, max_word_len)
        x_char_ids_reshaped = x_char_ids.view(-1, max_word_len)

        # Embed characters: (batch_size * seq_len, max_word_len, char_embedding_dim)
        char_emb = self.char_embedding(x_char_ids_reshaped)
        char_emb = self.dropout(char_emb) # Apply dropout to character embeddings

        # Permute for Conv1d: (batch_size * seq_len, char_embedding_dim, max_word_len)
        char_emb_permuted = char_emb.permute(0, 2, 1)

        # Convolution: (batch_size * seq_len, char_cnn_out_channels, convolved_len)
        # The convolved_len depends on max_word_len, kernel_size, and padding.
        # Example: if max_word_len=16, kernel=5, padding=0 -> convolved_len = 16-5+1 = 12
        # Example: if max_word_len=16, kernel=5, padding=2 -> convolved_len = 16-5+1+2*2 = 16
        char_conv_out = self.conv1d(char_emb_permuted)
        char_conv_out = F.relu(char_conv_out) # Apply ReLU

        # Max-pool over the convolved length dimension: (batch_size * seq_len, char_cnn_out_channels)
        # The kernel_size for max_pool1d should be the full length of the convolved dimension.
        char_pooled = F.max_pool1d(char_conv_out, kernel_size=char_conv_out.shape[2]).squeeze(2)

        # Reshape back to (batch_size, seq_len, char_cnn_out_channels)
        final_char_emb = char_pooled.view(batch_size, seq_len, -1)

        return final_char_emb


# HighwayNetwork class remains the same as provided in the previous full model response
class HighwayNetwork(nn.Module):
    def __init__(self, input_dim, num_layers):
        super(HighwayNetwork, self).__init__()
        self.num_layers = num_layers
        self.transform_gates = nn.ModuleList([nn.Linear(input_dim, input_dim) for _ in range(num_layers)])
        self.normal_layers = nn.ModuleList([nn.Linear(input_dim, input_dim) for _ in range(num_layers)])

    def forward(self, x):
        for i in range(self.num_layers):
            transform_gate_output = torch.sigmoid(self.transform_gates[i](x))
            normal_layer_output = F.relu(self.normal_layers[i](x))
            x = transform_gate_output * normal_layer_output + (1 - transform_gate_output) * x
        return x

# BiDAFAttention class remains the same as provided in the previous full model response
class BiDAFAttention(nn.Module):
    def __init__(self, hidden_size_times_2d): # effectively 2 * d, e.g., 2 * 300 = 600
        super(BiDAFAttention, self).__init__()
        self.hidden_size_times_2d = hidden_size_times_2d
        self.similarity_weight = nn.Linear(self.hidden_size_times_2d * 3, 1, bias=False)

    def forward(self, C_contextual, Q_contextual, C_mask, Q_mask):
        # C_contextual: (batch, C_len, 2d)
        # Q_contextual: (batch, Q_len, 2d)
        batch_size, C_len, _ = C_contextual.shape
        _, Q_len, _ = Q_contextual.shape

        C_expanded = C_contextual.unsqueeze(2).expand(-1, -1, Q_len, -1)
        Q_expanded = Q_contextual.unsqueeze(1).expand(-1, C_len, -1, -1)
        elementwise_prod = C_expanded * Q_expanded
        concat_features = torch.cat((C_expanded, Q_expanded, elementwise_prod), dim=3)
        S = self.similarity_weight(concat_features).squeeze(3)

        S_masked_q = S.masked_fill(Q_mask.unsqueeze(1) == 0, -float('inf'))
        S_masked_c = S.masked_fill(C_mask.unsqueeze(2) == 0, -float('inf'))

        alpha = F.softmax(S_masked_q, dim=2)
        A = torch.bmm(alpha, Q_contextual)

        max_S_c = torch.max(S, dim=2)[0]
        max_S_c_masked = max_S_c.masked_fill(C_mask == 0, -float('inf'))
        b_weights = F.softmax(max_S_c_masked, dim=1)
        C_prime = torch.bmm(b_weights.unsqueeze(1), C_contextual).squeeze(1)
        B = C_prime.unsqueeze(1).expand(-1, C_len, -1)

        g_c_a = C_contextual * A
        g_c_b = C_contextual * B
        G = torch.cat((C_contextual, A, g_c_a, g_c_b), dim=2)
        return G


class BiDAF(nn.Module):
    def __init__(self,
                 # Word embedding params
                 word_vocab_size, word_embedding_dim, pretrained_word_embeddings, word_padding_idx,
                 # Character embedding params
                 char_vocab_size, char_embedding_dim, char_cnn_out_channels,
                 char_cnn_kernel_size, char_padding_idx,
                 # Model general params
                 hidden_size, # This is 'd' in the paper, e.g., 300
                 num_highway_layers=2, dropout_rate=0.2):
        super(BiDAF, self).__init__()

        self.word_embedding_dim = word_embedding_dim
        self.char_output_dim = char_cnn_out_channels
        self.combined_embedding_dim = self.word_embedding_dim + self.char_output_dim # e.g., 300 + 100 = 400

        self.hidden_size = hidden_size # d (e.g., 300)
        self.dropout_rate = dropout_rate
        self.word_padding_idx = word_padding_idx

        # 1. Word Embedding Layer
        self.word_embedding = nn.Embedding.from_pretrained(
            embeddings=pretrained_word_embeddings,
            freeze=False, # Set to False to fine-tune GloVe embeddings
            padding_idx=self.word_padding_idx
        )

        # 1b. Character Embedding Layer
        self.char_embedding_layer = CharEmbedding(
            char_vocab_size=char_vocab_size,
            char_embedding_dim=char_embedding_dim, # e.g., 8 or 20
            char_cnn_out_channels=self.char_output_dim, # e.g., 100
            char_cnn_kernel_size=char_cnn_kernel_size, # e.g., 5
            char_padding_idx=char_padding_idx,
            dropout_rate=self.dropout_rate
        )

        # 2. Highway Network
        # Input is the concatenated word and char embeddings
        self.highway_network = HighwayNetwork(input_dim=self.combined_embedding_dim, num_layers=num_highway_layers)

        # 3. Contextual Embedding Layer (BiLSTM)
        # Input: combined_embedding_dim (e.g., 400)
        # Output: 2 * hidden_size (e.g., 2 * 300 = 600) because bidirectional
        self.contextual_lstm = nn.LSTM(
            input_size=self.combined_embedding_dim,
            hidden_size=self.hidden_size, # d
            num_layers=1,
            bidirectional=True,
            batch_first=True,
            dropout=0 # Dropout between LSTM layers only if num_layers > 1; we apply manually after
        )

        # 4. Attention Flow Layer
        # Input features to attention are of size 2*hidden_size (output of contextual_lstm)
        self.attention = BiDAFAttention(hidden_size_times_2d=2 * self.hidden_size)

        # 5. Modeling Layer (BiLSTM)
        # Input: 8 * hidden_size (output of attention G, since G contains 4 components each of size 2*hidden_size)
        # Output: 2 * hidden_size
        self.modeling_lstm = nn.LSTM(
            input_size=8 * self.hidden_size, # Each of the 4 components in G is 2*hidden_size
            hidden_size=self.hidden_size, # d
            num_layers=2, # As per paper
            bidirectional=True,
            batch_first=True,
            dropout=self.dropout_rate if 2 > 1 else 0 # LSTM internal dropout
        )

        # 6. Output Layer
        # Predicts start and end logits
        # Input for prediction: concat(G, M) -> (8*hidden_size + 2*hidden_size) = 10 * hidden_size
        self.start_output_linear = nn.Linear(10 * self.hidden_size, 1)
        self.end_output_linear = nn.Linear(10 * self.hidden_size, 1)

        self.dropout_layer = nn.Dropout(self.dropout_rate)


    def forward(self, context_word_ids, question_word_ids, context_char_ids, question_char_ids):
        # context_word_ids: (batch, C_len)
        # question_word_ids: (batch, Q_len)
        # context_char_ids: (batch, C_len, max_word_len)
        # question_char_ids: (batch, Q_len, max_word_len)

        # Masks (1 for tokens, 0 for padding)
        C_mask = (context_word_ids != self.word_padding_idx)
        Q_mask = (question_word_ids != self.word_padding_idx)

        # 1a. Word Embedding
        C_word_emb = self.word_embedding(context_word_ids)    # (batch, C_len, word_emb_dim)
        Q_word_emb = self.word_embedding(question_word_ids)  # (batch, Q_len, word_emb_dim)

        # 1b. Character Embedding
        C_char_emb = self.char_embedding_layer(context_char_ids) # (batch, C_len, char_output_dim)
        Q_char_emb = self.char_embedding_layer(question_char_ids)# (batch, Q_len, char_output_dim)

        # Concatenate word and char embeddings
        C_emb_combined = torch.cat((C_word_emb, C_char_emb), dim=2) # (batch, C_len, combined_emb_dim)
        Q_emb_combined = torch.cat((Q_word_emb, Q_char_emb), dim=2) # (batch, Q_len, combined_emb_dim)

        C_emb_combined = self.dropout_layer(C_emb_combined)
        Q_emb_combined = self.dropout_layer(Q_emb_combined)

        # 2. Highway Network
        C_highway = self.highway_network(C_emb_combined)     # (batch, C_len, combined_emb_dim)
        Q_highway = self.highway_network(Q_emb_combined)   # (batch, Q_len, combined_emb_dim)

        # 3. Contextual Embedding Layer (BiLSTM)
        # Input: combined_emb_dim, Output: (batch, seq_len, 2*hidden_size)
        C_contextual, _ = self.contextual_lstm(C_highway)
        Q_contextual, _ = self.contextual_lstm(Q_highway)

        C_contextual = self.dropout_layer(C_contextual)
        Q_contextual = self.dropout_layer(Q_contextual)

        # 4. Attention Flow Layer
        # G: (batch, C_len, 8*hidden_size)
        G = self.attention(C_contextual, Q_contextual, C_mask.float(), Q_mask.float())
        G = self.dropout_layer(G) # Dropout after attention output

        # 5. Modeling Layer (BiLSTM)
        # M: (batch, C_len, 2*hidden_size)
        M, _ = self.modeling_lstm(G)
        M = self.dropout_layer(M) # Dropout after modeling layer output

        # 6. Output Layer
        output_features = torch.cat((G, M), dim=2) # (batch, C_len, 10*hidden_size)

        start_logits = self.start_output_linear(output_features).squeeze(2) # (batch, C_len)
        end_logits = self.end_output_linear(output_features).squeeze(2)   # (batch, C_len)

        start_logits_masked = start_logits.masked_fill(C_mask == 0, -float('inf'))
        end_logits_masked = end_logits.masked_fill(C_mask == 0, -float('inf'))

        return start_logits_masked, end_logits_masked




import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from datasets import Dataset, load_from_disk
import json
import numpy as np
import os
from tqdm import tqdm # For progress bars

# --- Configuration & Hyperparameters ---
# Paths (ensure these match your saved files)
PROCESSED_TRAIN_PATH = "./squad_train_processed_char.hf"
PROCESSED_VALIDATION_PATH = "./squad_validation_processed_char.hf"
WORD_VOCAB_PATH = "./squad_vocab_spacy.json"
CHAR_VOCAB_PATH = "./squad_char_vocab.json"
PRETRAINED_WORD_EMB_PATH = "./squad_embedding_matrix_spacy.pt" # GloVe matrix

# Model & Embedding Hyperparameters (should match your model definition)
# Word Embeddings
WORD_EMBEDDING_DIM = 300 # User specified
# Character Embeddings (based on BiDAF paper / common practice)
CHAR_EMBEDDING_DIM = 8    # Dimension of individual char embedding
CHAR_CNN_OUT_CHANNELS = 50 # Output dim of char CNN, effectively char-level word emb dim
CHAR_CNN_KERNEL_SIZE = 5
# Model Structure
HIDDEN_SIZE = 128 # 'd' in BiDAF paper, often same as word_embedding_dim
NUM_HIGHWAY_LAYERS = 2
DROPOUT_RATE = 0.2
# MAX_WORD_LEN = 25 # Defined during preprocessing

# Training Hyperparameters
LEARNING_RATE = 0.001
BATCH_SIZE = 24 # Adjust based on your GPU memory
NUM_EPOCHS = 10 # Start with a few, increase later
CLIP_GRAD_NORM = 5.0 # For gradient clipping

# --- Device Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- Load Vocabularies and Pretrained Embeddings ---
try:
    with open(WORD_VOCAB_PATH, 'r') as f:
        word_vocab_data = json.load(f)
        word_to_idx = word_vocab_data['word_to_idx']
    WORD_VOCAB_SIZE = len(word_to_idx)
    WORD_PADDING_IDX = word_to_idx.get("<PAD>", 0) # Ensure your PAD token name
    print(f"Word vocabulary loaded. Size: {WORD_VOCAB_SIZE}, PAD_IDX: {WORD_PADDING_IDX}")

    with open(CHAR_VOCAB_PATH, 'r') as f:
        char_vocab_data = json.load(f)
        char_to_idx = char_vocab_data['char_to_idx']
    CHAR_VOCAB_SIZE = len(char_to_idx)
    CHAR_PADDING_IDX = char_to_idx.get("<CHAR_PAD>", 0) # Ensure your CHAR_PAD token name
    print(f"Character vocabulary loaded. Size: {CHAR_VOCAB_SIZE}, CHAR_PAD_IDX: {CHAR_PADDING_IDX}")

    pretrained_word_embeddings = torch.load(PRETRAINED_WORD_EMB_PATH, map_location=device)
    # Verify embedding dim if necessary, should be WORD_EMBEDDING_DIM
    print(f"Pretrained word embeddings loaded. Shape: {pretrained_word_embeddings.shape}")
    if pretrained_word_embeddings.shape[1] != WORD_EMBEDDING_DIM:
        raise ValueError(f"Pretrained embedding dim ({pretrained_word_embeddings.shape[1]}) != WORD_EMBEDDING_DIM ({WORD_EMBEDDING_DIM})")

except FileNotFoundError as e:
    print(f"Error loading prerequisite file: {e}. Please ensure all preprocessing steps are complete.")
    exit()
except KeyError as e:
    print(f"Error: Special token {e} not found in loaded vocabularies.")
    exit()


# --- Load Processed Datasets ---
try:
    train_dataset_processed = load_from_disk(PROCESSED_TRAIN_PATH, keep_in_memory=True)
    val_dataset_processed = load_from_disk(PROCESSED_VALIDATION_PATH, keep_in_memory=True)
except Exception as e: # More generic catch for Hugging Face load_from_disk issues
    print(f"Error loading processed datasets: {e}")
    print("Please ensure your processed data paths are correct and data exists.")
    exit()

# Set format for PyTorch
columns_to_torch = ['context_ids', 'question_ids', 'context_char_ids', 'question_char_ids',
                    'token_answer_start', 'token_answer_end']
train_dataset_processed.set_format(type='torch', columns=columns_to_torch)
val_dataset_processed.set_format(type='torch', columns=columns_to_torch)


# --- DataLoaders ---
train_dataloader = DataLoader(train_dataset_processed, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset_processed, batch_size=BATCH_SIZE, shuffle=False) # No shuffle for validation

print(f"DataLoaders created. Train batches: {len(train_dataloader)}, Val batches: {len(val_dataloader)}")

Using device: cuda
Word vocabulary loaded. Size: 103963, PAD_IDX: 0
Character vocabulary loaded. Size: 1374, CHAR_PAD_IDX: 0
Pretrained word embeddings loaded. Shape: torch.Size([103963, 300])


Loading dataset from disk:   0%|          | 0/22 [00:00<?, ?it/s]

DataLoaders created. Train batches: 3650, Val batches: 441


In [2]:
# --- Model Instantiation (Ensure your BiDAF class definition is available) ---
# from your_model_file import BiDAF # Assuming BiDAF class is in another file
# For this script, ensure BiDAF class with CharEmbedding is defined above or imported.
# Make sure the BiDAF class definition from the previous step is available here.
# (The full BiDAF class with CharEmbedding from the previous response should be here)

# Re-paste the CharEmbedding, HighwayNetwork, BiDAFAttention, and BiDAF classes here if not imported
# For brevity, I'll assume they are defined above this point in your script.
# Make sure the BiDAF class from the previous response (with character embeddings) is defined here
# For example:
# class CharEmbedding(nn.Module): ...
# class HighwayNetwork(nn.Module): ...
# class BiDAFAttention(nn.Module): ...
# class BiDAF(nn.Module): ... (the one that takes char_... params)

model = BiDAF(
    word_vocab_size=WORD_VOCAB_SIZE,
    word_embedding_dim=WORD_EMBEDDING_DIM,
    pretrained_word_embeddings=pretrained_word_embeddings,
    word_padding_idx=WORD_PADDING_IDX,
    char_vocab_size=CHAR_VOCAB_SIZE,
    char_embedding_dim=CHAR_EMBEDDING_DIM,
    char_cnn_out_channels=CHAR_CNN_OUT_CHANNELS,
    char_cnn_kernel_size=CHAR_CNN_KERNEL_SIZE,
    char_padding_idx=CHAR_PADDING_IDX,
    hidden_size=HIDDEN_SIZE,
    num_highway_layers=NUM_HIGHWAY_LAYERS,
    dropout_rate=DROPOUT_RATE
).to(device)

print("BiDAF Model instantiated and moved to device.")

# --- Loss Function and Optimizer ---
criterion = nn.CrossEntropyLoss(ignore_index=-1) # Use -1 if your targets might have it for unanswerable, though SQuAD 1.1 shouldn't
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)


# --- Metrics Calculation (Simplified for illustration) ---
def compute_metrics(pred_starts, pred_ends, true_starts, true_ends):
    # This is a very simplified token-level EM.
    # True SQuAD EM/F1 requires text normalization and comparison.
    em_sum = 0
    f1_sum = 0
    num_examples = len(pred_starts)

    for i in range(num_examples):
        ps, pe = pred_starts[i], pred_ends[i]
        ts, te = true_starts[i], true_ends[i]

        # Exact Match (token level)
        if ps == ts and pe == te:
            em_sum += 1

        # F1 Score (token level)
        pred_tokens = set(range(ps, pe + 1))
        true_tokens = set(range(ts, te + 1))

        common_tokens = len(pred_tokens.intersection(true_tokens))
        if common_tokens == 0:
            f1 = 0.0
        else:
            precision = common_tokens / len(pred_tokens)
            recall = common_tokens / len(true_tokens)
            f1 = (2 * precision * recall) / (precision + recall)
        f1_sum += f1

    return (em_sum / num_examples) * 100, (f1_sum / num_examples) * 100


# --- Training Loop ---
def train_epoch(model, dataloader, optimizer, criterion, device, clip_norm):
    model.train()
    epoch_loss = 0
    progress_bar = tqdm(dataloader, desc="Training", leave=False)

    for batch in progress_bar:
        context_ids = batch['context_ids'].to(device)
        question_ids = batch['question_ids'].to(device)
        context_char_ids = batch['context_char_ids'].to(device)
        question_char_ids = batch['question_char_ids'].to(device)
        true_start_idx = batch['token_answer_start'].to(device)
        true_end_idx = batch['token_answer_end'].to(device)

        optimizer.zero_grad()

        # Inside train_epoch, before model call:
        if context_ids.max().item() >= WORD_VOCAB_SIZE or context_ids.min().item() < 0:
            print("Error: context_word_ids out of bounds for word vocab!")
        if context_char_ids.max().item() >= CHAR_VOCAB_SIZE or context_char_ids.min().item() < 0:
            print("Error: context_char_ids out of bounds for char vocab!")
        # Similar checks for question_ids and question_char_ids

        start_logits, end_logits = model(context_word_ids=context_ids,
                                         question_word_ids=question_ids,
                                         context_char_ids=context_char_ids,
                                         question_char_ids=question_char_ids)

        if not ((true_start_idx >= 0) & (true_start_idx < start_logits.size(1))).all():
            print("Error: true_start_idx out of bounds!")
            print("Min start_idx:", true_start_idx.min().item(), "Max start_idx:", true_start_idx.max().item())
            print("Logits shape (num_classes for start):", start_logits.shape)
            # Optionally print the problematic indices/batch
            problem_indices_start = ~((true_start_idx >= 0) & (true_start_idx < start_logits.size(1)))
            print("Problematic true_start_idx:", true_start_idx[problem_indices_start])
            # You might want to raise an error or use pdb here to inspect the batch

        if not ((true_end_idx >= 0) & (true_end_idx < end_logits.size(1))).all():
            print("Error: true_end_idx out of bounds!")
            print("Min end_idx:", true_end_idx.min().item(), "Max end_idx:", true_end_idx.max().item())
            print("Logits shape (num_classes for end):", end_logits.shape)
            problem_indices_end = ~((true_end_idx >= 0) & (true_end_idx < end_logits.size(1)))
            print("Problematic true_end_idx:", true_end_idx[problem_indices_end])

        # Ensure target indices are valid (not -1 if that was used for unmappable)
        # For SQuAD v1.1, start/end should always be valid.
        # If there's a possibility of -1 from preprocessing issues:
        # valid_indices = (true_start_idx != -1) & (true_end_idx != -1)
        # start_logits = start_logits[valid_indices]
        # end_logits = end_logits[valid_indices]
        # true_start_idx = true_start_idx[valid_indices]
        # true_end_idx = true_end_idx[valid_indices]
        # if not valid_indices.any(): continue

        loss_start = criterion(start_logits, true_start_idx)
        loss_end = criterion(end_logits, true_end_idx)
        total_loss = loss_start + loss_end

        total_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_norm) # Gradient clipping
        optimizer.step()

        epoch_loss += total_loss.item()
        progress_bar.set_postfix({'loss': total_loss.item()})

    return epoch_loss / len(dataloader)


# --- Evaluation Loop ---
def evaluate(model, dataloader, criterion, device):
    model.eval()
    epoch_loss = 0
    all_pred_starts, all_pred_ends = [], []
    all_true_starts, all_true_ends = [], []

    progress_bar = tqdm(dataloader, desc="Evaluating", leave=False)

    with torch.no_grad():
        for batch in progress_bar:
            context_ids = batch['context_ids'].to(device)
            question_ids = batch['question_ids'].to(device)
            context_char_ids = batch['context_char_ids'].to(device)
            question_char_ids = batch['question_char_ids'].to(device)
            true_start_idx = batch['token_answer_start'].to(device)
            true_end_idx = batch['token_answer_end'].to(device)

            start_logits, end_logits = model(context_word_ids=context_ids,
                                             question_word_ids=question_ids,
                                             context_char_ids=context_char_ids,
                                             question_char_ids=question_char_ids)

            loss_start = criterion(start_logits, true_start_idx)
            loss_end = criterion(end_logits, true_end_idx)
            total_loss = loss_start + loss_end
            epoch_loss += total_loss.item()

            pred_start_batch = torch.argmax(start_logits, dim=1)
            pred_end_batch = torch.argmax(end_logits, dim=1)

            all_pred_starts.extend(pred_start_batch.cpu().tolist())
            all_pred_ends.extend(pred_end_batch.cpu().tolist())
            all_true_starts.extend(true_start_idx.cpu().tolist())
            all_true_ends.extend(true_end_idx.cpu().tolist())

            progress_bar.set_postfix({'loss': total_loss.item()})

    avg_loss = epoch_loss / len(dataloader)
    em, f1 = compute_metrics(all_pred_starts, all_pred_ends, all_true_starts, all_true_ends)

    return avg_loss, em, f1


# --- Main Training Orchestration ---
best_val_f1 = -1.0
model_save_path = "./bidaf_best_model.pt"

print("\nStarting training...")
for epoch in range(NUM_EPOCHS):
    train_loss = train_epoch(model, train_dataloader, optimizer, criterion, device, CLIP_GRAD_NORM)
    val_loss, val_em, val_f1 = evaluate(model, val_dataloader, criterion, device)

    if scheduler:
        scheduler.step(val_loss) # Or scheduler.step(val_f1) if using F1

    print(f"Epoch {epoch+1}/{NUM_EPOCHS}:")
    print(f"\tTrain Loss: {train_loss:.4f}")
    print(f"\tVal Loss: {val_loss:.4f} | Val EM: {val_em:.2f}% | Val F1: {val_f1:.2f}%")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), model_save_path)
        print(f"\tNew best model saved to {model_save_path} (F1: {best_val_f1:.2f}%)")

print("\nTraining complete.")
print(f"Best Validation F1: {best_val_f1:.2f}% (Model saved at {model_save_path})")

# To load the best model later:
# model.load_state_dict(torch.load(model_save_path))
# model.eval()

BiDAF Model instantiated and moved to device.


C:\Users\User\ML\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(



Starting training...


Training:   1%|          | 30/3650 [00:13<24:20,  2.48it/s, loss=8.86] 

Error: true_start_idx out of bounds!
Min start_idx: -1 Max start_idx: 175
Logits shape (num_classes for start): torch.Size([24, 512])
Problematic true_start_idx: tensor([-1], device='cuda:0')
Error: true_end_idx out of bounds!
Min end_idx: -1 Max end_idx: 176
Logits shape (num_classes for end): torch.Size([24, 512])
Problematic true_end_idx: tensor([-1], device='cuda:0')


...

Training:  62%|██████▏   | 2258/3650 [14:42<09:00,  2.58it/s, loss=1.03] 

Error: true_start_idx out of bounds!
Min start_idx: -1 Max start_idx: 184
Logits shape (num_classes for start): torch.Size([24, 512])
Problematic true_start_idx: tensor([-1], device='cuda:0')
Error: true_end_idx out of bounds!
Min end_idx: -1 Max end_idx: 186
Logits shape (num_classes for end): torch.Size([24, 512])
Problematic true_end_idx: tensor([-1], device='cuda:0')


Training:  83%|████████▎ | 3024/3650 [19:39<04:03,  2.57it/s, loss=1.73] 

Error: true_start_idx out of bounds!
Min start_idx: -1 Max start_idx: 204
Logits shape (num_classes for start): torch.Size([24, 512])
Problematic true_start_idx: tensor([-1], device='cuda:0')
Error: true_end_idx out of bounds!
Min end_idx: -1 Max end_idx: 204
Logits shape (num_classes for end): torch.Size([24, 512])
Problematic true_end_idx: tensor([-1], device='cuda:0')


Epoch 10/10:
	Train Loss: 1.2436
	Val Loss: 4.0594 | Val EM: 42.86% | Val F1: 59.38%

Training complete.
Best Validation F1: 60.65% (Model saved at ./bidaf_best_model.pt)


In [3]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()